In [1]:
!conda info --envs

# conda environments:
#
                         /Users/titai/Library/r-miniconda
                         /Users/titai/Library/r-miniconda/envs/r-reticulate
base                     /Users/titai/miniconda3
annovar                  /Users/titai/miniconda3/envs/annovar
bmi7320                  /Users/titai/miniconda3/envs/bmi7320
bs                       /Users/titai/miniconda3/envs/bs
celloracle_py3.9.16      /Users/titai/miniconda3/envs/celloracle_py3.9.16
pcpp                     /Users/titai/miniconda3/envs/pcpp
pyscenic              *  /Users/titai/miniconda3/envs/pyscenic
scvi                     /Users/titai/miniconda3/envs/scvi
seacells                 /Users/titai/miniconda3/envs/seacells
tf                       /Users/titai/miniconda3/envs/tf



In [2]:
import os
import pandas as pd
import numpy as np
import scanpy as sc
import pyscenic
import loompy as lp

from MulticoreTSNE import MulticoreTSNE as TSNE

import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
!ls *.h5ad

hgsoc_integrated_only.h5ad    hgsoc_metacells_20230829.h5ad
hgsoc_metacells.h5ad          hgsoc_with_SEACell_ID.h5ad


In [4]:
#load the data
adata = sc.read_h5ad("hgsoc_metacells.h5ad")

In [5]:
adata

AnnData object with n_obs × n_vars = 1211 × 35019
    obs: 'GSE_ID', 'GSM_ID', 'Treatment Status', 'Clinical stage', 'Clinical stage (Major)', 'cell_type'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'highly_variable_nbatches', 'highly_variable_intersection'
    uns: 'GSE_ID_colors', 'Treatment Status_colors', 'cell_type_colors', 'hvg', 'log1p', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap'
    varm: 'PCs'
    layers: 'raw'
    obsp: 'connectivities', 'distances'

In [6]:
sc.pp.filter_genes(adata, min_cells=(len(adata.obs)/2))

In [7]:
adata

AnnData object with n_obs × n_vars = 1211 × 10504
    obs: 'GSE_ID', 'GSM_ID', 'Treatment Status', 'Clinical stage', 'Clinical stage (Major)', 'cell_type'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'highly_variable_nbatches', 'highly_variable_intersection', 'n_cells'
    uns: 'GSE_ID_colors', 'Treatment Status_colors', 'cell_type_colors', 'hvg', 'log1p', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap'
    varm: 'PCs'
    layers: 'raw'
    obsp: 'connectivities', 'distances'

In [8]:
adata.X

<1211x10504 sparse matrix of type '<class 'numpy.float64'>'
	with 10638971 stored elements in Compressed Sparse Row format>

In [9]:
adata.layers["raw"]

<1211x10504 sparse matrix of type '<class 'numpy.float64'>'
	with 10638971 stored elements in Compressed Sparse Row format>

In [10]:
print(np.amax(adata.X.toarray()))
print(np.amax(adata.layers["raw"].toarray()))

12.09535074058112
447900.0


In [11]:
adata.obs['Treatment Status'].unique()

['Control', 'Before chemotherapy', 'After chemotherapy']
Categories (3, object): ['After chemotherapy', 'Before chemotherapy', 'Control']

In [12]:
#Create a loom file
row_attrs = {
"Gene": np.array(adata.var_names),
}
col_attrs = {
"CellID": np.array(adata.obs_names),
"nGene": np.array(np.sum(adata.layers["raw"].transpose()>0, axis=0)).flatten(), 
"nUMI": np.array(np.sum(adata.layers["raw"].transpose(),axis=0)).flatten(),
}
lp.create("hgsoc_metacells_with_control_20230927.loom",adata.layers["raw"].transpose(),row_attrs, col_attrs)  ### change the .loom file name